In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%writefile .env

TAVILY_API_KEY = your_key
JINA_API_KEY = your_key
CHROME_INSTANCE_PATH = /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
BROWSER_HEADLESS=False


Overwriting .env


In [3]:
import sys
from src.workflow import run_agent_workflow

In [4]:
user_query = '''
    I would like to analyze Amazon product sales data.
    The target for analysis is the './data/Amazon_Sale_Report.csv' file.
    Please conduct an analysis to extract marketing insights based on this data.
    Please perform various analytical techniques starting from basic data attribute exploration, product sales trends, variable relationships, variable combinations, etc.
    If there are any additional analyses needed to extract insights after the data analysis, please perform those as well.
    Please include detailed analysis in the report along with supporting images and charts.
    Please save the final report in PDF format.
'''

user_query = '''
    이것은 아마존 상품판매 데이터를 분석하고 싶습니다.
    분석대상은 "./data/Amazon_Sale_Report.csv" 파일 입니다.
    데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요.
    분석은 기본적인 데이터 속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요.
    데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요.
    분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요.
    최종 리포트는 pdf 형태로 저장해 주세요.
'''

#user_query = '''
#    이것은 아마존 상품판매 데이터를 분석하고 싶습니다. 분석대상은 "./data/Amazon_Sale_Report.csv" 파일 입니다. 데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요. 분석은 간단하게 1개정도만 해 주세요. 코드 테스트 중이거든요 데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요. 분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요. 최종 리포트는 pdf 형태로 저장해 주세요.
#    '''

# user_query = '''
#     영문으로 작성된 pptx 파일을 한글로 변환하고 싶습니다.
#     대상은 "./data/bedrock.pptx" 파일 입니다.
#     슬라이드의 영문을 한글로 번역하고 슬라이드 structure는 유지해 주세요.
# '''

# user_query = '''
#     주어진 프로젝트 소스코드는 "Bedrock-Manus: AI automation framework optimized for Amazon Bedrock and business use cases 입니다."
#     프로젝트 소스코드는 "./data/08_bedrock_manus.tar.gz" 파일 입니다.
#     나는 Streamlit을 이용해서 이 소스코드에 대한 UI를 만들고 싶습니다. 
#     제가 원하는 요구사항은
#     1. 현재 스텝이 무엇인지 (coodinator, supervisor, planner 등등) 보이는 곳이 있으면 좋겠습니다.
#         - 스텝에 대한 자세한 정보는 "./README.md"를 참고하시면 됩니다. 
#     2. 각 스텝에서 진행되는 것이 보였으면 좋겠습니다. 즉, 각 스텝에서 llm이 생성하는 토큰들을 실시간으로 볼 수 있으면 좋겠습니다. 
#     3. "app.py" 파일만 생성하지 말고, 필요하다면 기존의 다른 파일 (workflow.py 등)들도  수정하세요. 
#     4. 수정된 파일은 모두 ./artifacts 에 넣어주세요.
#     참고할 사항이 있습니다. 
#     1. 소스코드 수행은 "main.py"로 수행됩니다. 
#     2. "./src/service/workflow_service.py" 이 파일은 참고하지 마세요. 

#     UI 생성을 위한 프로젝트 코드를 작성해주세요
#     결과물은 ./artifacts 에 넣어주세요.
# '''

# user_query = '''
#     "2025년 중국 가전 신제품 출시에 대한 리포트 작성해줘."
# '''

In [17]:
import os
import shutil

def remove_artifact_folder(folder_path="./artifacts/"):
    """
    ./artifact/ 폴더가 존재하면 삭제하는 함수
    
    Args:
        folder_path (str): 삭제할 폴더 경로
    """
    if os.path.exists(folder_path):
        print(f"'{folder_path}' 폴더를 삭제합니다...")
        try:
            # 폴더와 그 내용을 모두 삭제
            shutil.rmtree(folder_path)
            print(f"'{folder_path}' 폴더가 성공적으로 삭제되었습니다.")
        except Exception as e:
            print(f"오류 발생: {e}")
    else:
        print(f"'{folder_path}' 폴더가 존재하지 않습니다.")


import nest_asyncio
nest_asyncio.apply()

remove_artifact_folder()

result = run_agent_workflow(
    user_input=user_query,
    debug=False
)

# Print the conversation history
print("\n=== Conversation History ===")
print ("result", result)
for history in result["history"]:

    print ("===")
    print (f'agent: {history["agent"]}')
    print (f'message: {history["message"]}')


INFO [src.workflow] ===== Starting workflow =====

INFO [src.workflow] 
user input: 
    이것은 아마존 상품판매 데이터를 분석하고 싶습니다.
    분석대상은 "./data/Amazon_Sale_Report.csv" 파일 입니다.
    데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요.
    분석은 기본적인 데이터 속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요.
    데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요.
    분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요.
    최종 리포트는 pdf 형태로 저장해 주세요.


INFO [src.graph.nodes] ===== Coordinator talking...... =====

INFO [src.utils.strands_sdk_utils] COORDINATOR - Prompt Cache Disabled


'./artifacts/' 폴더가 존재하지 않습니다.
handoff_to_planner: I'll need to consult our planning system for this request.

This appears to be a complex data analysis request involving Amazon sales data that requires multiple steps including data exploration, trend analysis, variable relationship analysis, insight extraction, visualization, and PDF report generation. I'll pass this to our planning system for a detailed execution plan.


INFO [src.graph.nodes] ===== 여기서 무언가를 해도 되겠죠? =====

INFO [src.graph.nodes] 
Current state messages:
[ { 'content': [ { 'text': "[{'text': '\\nHere is a user request: <user_request>\\n    이것은 아마존 "
                           '상품판매 데이터를 분석하고 싶습니다.\\n    분석대상은 "./data/Amazon_Sale_Report.csv" 파일 '
                           '입니다.\\n    데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요.\\n    분석은 기본적인 데이터 '
                           '속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요.\\n    데이터 분석 후 '
                           '인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요.\\n    분석 리포트는 상세 분석과 그 것을 '
                           '뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요.\\n    최종 리포트는 pdf 형태로 저장해 '
                           "주세요.\\n</user_request>\\n'}]"}],
    'role': 'user'}]

INFO [src.graph.nodes] 
Coordinator response:
("handoff_to_planner: I'll need to consult our planning system for this request.\n"
 '\n'
 'This appears to be a complex data analysis request involving Amazon sales data that requires '

state {'goto': 'planner', 'history': [{'agent': 'coordinator', 'message': "handoff_to_planner: I'll need to consult our planning system for this request.\n\nThis appears to be a complex data analysis request involving Amazon sales data that requires multiple steps including data exploration, trend analysis, variable relationship analysis, insight extraction, visualization, and PDF report generation. I'll pass this to our planning system for a detailed execution plan."}]}

=== Conversation History ===
result GraphResult(status=<Status.COMPLETED: 'completed'>, results={'coordinator': NodeResult(result=MultiAgentResult(status=<Status.COMPLETED: 'completed'>, results={'coordinator': NodeResult(result=AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "handoff_to_planner: I'll need to consult our planning system for this request.\n\nThis appears to be a complex data analysis request involving Amazon sales data that requires multiple steps including data e

TypeError: 'GraphResult' object is not subscriptable